# Conversational AI - Chatbot

This Jupyter notebook demonstrates the implementation of a simple chatbot interface using multiple AI models (OpenAI GPT, Anthropic Claude, and DeepSeek). The notebook showcases how to:

- Set up and manage API keys for different AI services
- Create chat functions that handle messaging history and streaming responses
- Implement a Gradio-based user interface for real-time chat interaction
- Customize bot behavior using system messages
- Handle different scenarios in a retail context with specific sales promotions

The example includes a retail store assistant that provides information about sales and promotions, demonstrating how to create context-aware responses using system messages.

In [ ]:
# imports
import os
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
import gradio as gr

In [ ]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging
load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if deepseek_api_key:
    print(f"DeepSeek API Key found")
else:
    print("DeepSeek AP Key missing")
    

In [ ]:
# Initialize
openai = OpenAI()
claude = anthropic.Anthropic()
deepseek = OpenAI(api_key=deepseek_api_key, base_url="https://api.deepseek.com")

MODEL_OPENAI = 'gpt-4o-mini'
MODEL_CLAUDE = 'claude-3-haiku-20240307'
MODEL_DEEPSEEK = 'deepseek-reasoner' # deepseek-chat = deepseek V3 / deepseek-reasoner = deepseek R1

In [ ]:
system_message = "You are a helpful assistant"

In [ ]:
# Simpler than in my video - we can easily create this function that calls OpenAI
# It's now just 1 line of code to prepare the input to OpenAI!
def chat_openai(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    # You can see the construction happening behind the scenes
    print("History is:")
    print(history)
    print("And messages is:")
    print(messages)

    stream = openai.chat.completions.create(model=MODEL_OPENAI, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [ ]:
def chat_claude(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    result = claude.messages.stream(
        model=MODEL_CLAUDE,
        max_tokens=1000,
        temperature=0.7,
        system=system_message,
        messages=[
            {"role": "user", "content": message},
        ],
    )
    response = ""
    with result as stream:
        for text in stream.text_stream:
            response += text or ""
            yield response

In [ ]:
def chat_deepseek(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    stream = deepseek.chat.completions.create(model=MODEL_DEEPSEEK, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

## Gradio UI

In [ ]:
gr.ChatInterface(fn=chat_openai, type="messages").launch(pwa=True)

In [ ]:
system_message = "You are a helpful assistant in a clothes store. You should try to gently encourage \
the customer to try items that are on sale. Hats are 60% off, and most other items are 50% off. \
For example, if the customer says 'I'm looking to buy a hat', \
you could reply something like, 'Wonderful - we have lots of hats - including several that are part of our sales event.'\
Encourage the customer to buy hats if they are unsure what to get.\ In addition, you can encourage the customer to subscribe to our membership, as there is 10% off for new members "

In [ ]:
gr.ChatInterface(fn=chat_openai, type="messages").launch()

In [ ]:
system_message += "\nIf the customer asks for shoes, you should respond that shoes are not on sale today, \
but remind the customer to look at hats!"

In [ ]:
gr.ChatInterface(fn=chat_deepseek, type="messages").launch()

In [ ]:
items = ['belt', 'rings', 'necklaces']

In [ ]:
# Fixed a bug in this function brilliantly identified by student Gabor M.!
# I've also improved the structure of this function

def chat(message, history):

    relevant_system_message = system_message
    for item in items:  
        if item in message:
            relevant_system_message += f"The store does not sell {item}; if you are asked for {item}, be sure to point out other items on sale."
    
    messages = [{"role": "system", "content": relevant_system_message}] + history + [{"role": "user", "content": message}]

    stream = deepseek.chat.completions.create(model=MODEL_DEEPSEEK, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [ ]:
gr.ChatInterface(fn=chat_deepseek, type="messages").launch()